<a href="https://colab.research.google.com/github/MJMortensonWarwick/AAMA/blob/main/8_2_bayesian_methods_%26_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bayesian Methods and Machine Learning

Carrying on from our previous Notebook, we apply the Bayesian approach to a standard ML approach ... the linear regression. Ultimately both a frequentist and a Bayesian implementation have the same core goal in an ML framework - to build a model to estimate an unseen target value
($y$) from a set of features ($X$) - and most of elements of the workflow involved will remain the same. The key difference is in how they achieve this (i.e. the models).

The frequentist interpretation of linear regression is given by the following formula:<br><br>
$Y = \alpha + \beta X + e$<br><br>
... that is to say that given some set of $X$ values we can learn some parameters $\alpha$ and $\beta$ (one $\beta$ value for each feature/$X$) that can predict a corresponding target value ($Y$), subject to some error ($e$). Our goal is to find values for these parameters that minimise error at prediction time.

Let's compare this with the Bayesian interpretation:<br><br>
$Y = 𝓝\hspace{0.1cm}(\beta X + \sigma^2)$<br><br>
In this representation we consider $Y$ to be a normal distribution (our posterior) with $\mu = \beta X$ and some standard deviation ($\sigma^2$). We can think of $\sigma^2$ as basically the error term in the frequentist version .... that is that our line is fit to the values of $\alpha + \beta X$ and any gap between the line and the datapoints are the $\sigma^2$ of the model.

Further to this, while the values of $\alpha$ and $\beta$ are fixed values (point estimates) in the frequentist framework, for a Bayesian they are just parameters from an unknown distribution. Rather than finalising on a single value for each, instead we create a distribution of likely values each could take which, being a Gaussian distribution, will have a higher probability density around the mean ($\mu$).

At prediction we could use the mean of each parameter's distribution (i.e. the posterior distibutions for $\alpha$ and $\beta$) to make a single prediction - thus building a model that is a close approximation of the one a frequentist would build (at prediction time at least), but more likely we will make multiple predictions drawing samples from the distributions of each parameter. These multiple predictions can then be averaged to produce a single estimate for $Y$. In many ways we can consider the Bayesian approach, if executed in this fashion, as an ensemble model where we effectively build multiple models at prediction time by drawing different values for each parameter based on the probability densities created.

If this has all got a bit complicated, let's simplify as the following:
1. In a Bayesian model we don't treat our parameters as single values and instead create a probability distribution for each. In linear regression this means a probability distribution for $\alpha$ and for each $\beta$.
2. Rather than treating error as simply a by product of the model, we explicitly model error as another parameter ($\sigma^2$), and therefore as another probability distribution.
3. We learn the model by finding good distributions for each parameter using MCMC sampling (previous Notebook).
4. Rather than creating a single estimate for $Y$ we in fact create a probability distribution (the model's posterior) with multiple potential values, but with a probability mass centred around the mean of the distribution ($\mu$).
5. At prediction time the model works like an ensemble by making multiple predictions of the target ($Y$) using different parameters drawn from each parameter's posterior distribution. Our predictions can then be averaged to make a final prediction.

Let's see it in action! You need the file "reg.csv" from my.wbs.


In [ ]:
from google.colab import files
up = files.upload()

In [ ]:
import pandas as pd

df = pd.read_csv('reg.csv')

X = df[['x0', 'x1', 'x2']].to_numpy()
y = df[['Y']].to_numpy().ravel()

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

To this point we haven't done anything we haven't before. We've imported a dataset of three features ($X_0, X_1$ and $X_2$), and a set of target values ($Y$). We've split these into training and test (80/20).

In [ ]:
import pymc as pm
import arviz as az

with pm.Model() as bayes_LR:
    a = pm.Normal('intercept', 0, 1)
    b0 = pm.Normal('coeff0', 0, 1)
    b1 = pm.Normal('coeff1', 0, 1)
    b2 = pm.Normal('coeff2', 0, 1)
    s = pm.Exponential('error', 0.01)

    x0_ = pm.Data('features0', X_train[:, 0], mutable=True)
    x1_ = pm.Data('features1', X_train[:, 1], mutable=True)
    x2_ = pm.Data('features2', X_train[:, 2], mutable=True)

    obs = pm.Normal('observation', a + b0*x0_ + b1*x1_ + b2*x2_, s, observed=Y_train, shape=x0_.shape)

    trace = pm.sample(5000, random_seed=123, return_inferencedata=True)

Here we have specified our model. Much of this is as with the previous Notebook but obviously updated to our linear regression problem. We have specified $\alpha$ and $\beta$ as non-informative Gaussian distributions (with $\mu=0$ and $\sigma^2$ = 1).

For the error(s) we have used an exponential distribution. Error ($\sigma^2$) must be a positive value (negative error makes no sense) so we need a distribution that only allows this. The exponential fits that bill, although other contenders would be the gamma or a half-normal. The exponential takes a single parameter ($\lambda$) which determines the length of the tail. A lower value, as we have chosen, suggests a low amount of error (wishful thinking maybe).

After this we have split our features into three separate data containers (_pm.Data_). This allows us to later replace the data in the containers with our test data to make predictions.

Finally, we model $Y$ as a Gaussian with the regression formula as $\mu$ and our error variable as $\sigma^2$. We also pass the generated data as 'observed' (same as with the work in the previous Notebook).

Once the model is fit, again using _pm.Sample_ as we did previously, we can inspect the posteriors:

In [ ]:
with bayes_LR:
    print("Summary DF")
    print(az.summary(trace))
    print("\n")
    print("HDI")
    print(az.plot_posterior(trace, kind="hist"));

These outputs are the same as we saw previously, just there are more of them as we have a lot more distributions to estimate. Again our model has good $\hat{R}$ values and no obvious problems. We have mean estimates for each of our $\alpha$ and $\beta$ parameters. Most of our $\beta$ weights our model is fairly condifent about (narrow peaks and only about 0.3-0.4 variation in our 94% HDI) but $\alpha$ is a little less certain and not insubstantial error.

With the model specified / parameters learned, we can start predicting:

In [ ]:
with bayes_LR:
    pm.set_data({'features0': X_test[:, 0], 'features1': X_test[:, 1], 'features2': X_test[:, 2]})
    posterior = pm.sample_posterior_predictive(trace);

Here we return to our model and update the data containers with the values in _X\_test._ After this we can use our posterior in predictive mode.

In [ ]:
import pandas as pd

y_hat = posterior.posterior_predictive['observation'].mean(('chain', 'draw'))
y_pred = pd.DataFrame(y_hat)
y_pred.head(10)

This code should be fairly self-explanatory. We add the generated predictions into a dataframe. Our dataframe is 100 rows (although we print only 10), representing our average predicting for X\_test. However, these numbers (our $\hat{Y}$ predictions) are only really meaningful in comparison to the real $Y$ values.

Let's buid a dataframe to compare the two:

In [ ]:
y_pred["Pred"] = [round(i, 4) for i in y_hat.values]
y_pred["Real"] = Y_test
y_pred["SE"] = round((y_pred["Real"] - y_pred["Pred"]) **2, 4)
y_pred[["Real", "Pred", "SE"]].head(10)

Overall our predictions look pretty good. Some error, but we saw this in our error distribution ($\sigma^2$) earlier.

With these metrics calculated we can get the RMSE for our model:

In [ ]:
import math

rmse = math.sqrt(y_pred.SE.mean())
print(f'RMSE (Bayesian model) = {round(rmse, 4)}')

##TL;DR
We have seen a Bayesian implementation of linear regression and some of the benefits it can bring. We have fitted a model that seemingly predicts fairly well (only 14.5 $RMSE$) all the while we have Bayesian estimates around our parameters and 94% HDIs on what our parameters might be. I.e. we have a model that performs as it should, and we also have some degree of knowledge about how uncertain our model is, and potential other parameters. Good work! 😗